In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

In [2]:
import hyperspy.api as hs

In [3]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [4]:
path = '/Users/josephvas/Dropbox/ER-C/Projects/EMCD_YIG/EMCD_PCA/'
file_name1 = 'STEM_SI_C-_PCA.dm4'
file_name2 = 'STEM_SI_C+_PCA.dm4'

In [5]:
eels_data_minus = hs.load(path+file_name1)
eels_data_plus = hs.load(path+file_name2)

In [6]:
eels_data_minus.plot()

In [7]:
eels_data_minus.remove_background()

Qt WebEngine seems to be initialized from a plugin. Please set Qt::AA_ShareOpenGLContexts using QCoreApplication::setAttribute before constructing QGuiApplication.


/Users/josephvas/opt/anaconda3/lib/python3.9/site-packages/hyperspy/_components/power_law.py:151: RuntimeWarning: invalid value encountered in log
  r = 2 * (np.log(I1) - np.log(I2)) / (np.log(x2) - np.log(x1))


In [8]:
eels_data_plus.plot()
eels_data_plus.remove_background()

/Users/josephvas/opt/anaconda3/lib/python3.9/site-packages/hyperspy/_components/power_law.py:151: RuntimeWarning: invalid value encountered in log
  r = 2 * (np.log(I1) - np.log(I2)) / (np.log(x2) - np.log(x1))


eels_data_plus.plot()
eels_data_minus.plot()

In [9]:
# correcting the postedge
eels_minus_corrected = eels_data_minus.deepcopy()
eels_plus_corrected = eels_data_plus.deepcopy()

m= eels_minus_corrected.data.shape[0]
n = eels_minus_corrected.data.shape[1]

for i in range(0,n):
    for j in range(0,m):
        mean_plus = np.mean(eels_data_plus.inav[i,j].isig[740.:790.].data)
        mean_minus = np.mean(eels_data_minus.inav[i,j].isig[740.:790.].data)
        max_plus = np.max(eels_data_plus.inav[i,j].isig[:].data)
        max_minus = np.max(eels_data_plus.inav[i,j].isig[:].data)
        correction_factor = mean_plus/mean_minus
        eels_minus_corrected.data[j,i] = eels_minus_corrected.data[j,i]*correction_factor/max_plus
        eels_plus_corrected.data[j,i] = eels_plus_corrected.data[j,i]/max_plus

In [10]:
# substrat
eels_subtracted = eels_data_minus.deepcopy()

eels_subtracted = eels_plus_corrected-eels_minus_corrected

In [11]:
eels_subtracted.plot()


In [50]:
eels_plus_corrected.plot()
eels_minus_corrected.plot()

In [68]:
eels_subtracted.data.shape[0]

57

In [69]:
output_matrix = np.zeros([eels_subtracted.data.shape[0],eels_subtracted.data.shape[1],2])

In [80]:
#reconstructing the energy axis

slope = eels_subtracted.axes_manager[2].scale
c = eels_subtracted.axes_manager[2].offset
energy_axis = np.linspace(0,eels_subtracted.axes_manager[2].size-1,eels_subtracted.axes_manager[2].size)
energy_axis = slope*energy_axis+c

#defining the double gausisian to fit

# Define the model function for a single Gaussian curve
def gaussian(x, amp, mean, std_dev):
    return amp * np.exp(-(x - mean) ** 2 / (2 * std_dev ** 2))

# Define the model function for two Gaussian curves
def double_gaussian(x, amp1, mean1, std_dev1, amp2, mean2, std_dev2):
    return gaussian(x, amp1, mean1, std_dev1) + gaussian(x, amp2, mean2, std_dev2)

#initial settings
amp1 = 1
mean1 =710
std_dev1 = 5

amp2 = 1
mean2 = 724
std_dev2 = 5

initial_guess = [amp1, mean1, std_dev1, amp2, mean2, std_dev2]
output_matrix = np.zeros([eels_subtracted.data.shape[0],eels_subtracted.data.shape[1],2])


for i in range(0,n):
    for j in range(0,m):
        y_data =eels_subtracted.inav[i,j].isig[:].data
        params, covariance = curve_fit(double_gaussian, energy_axis, y_data, p0=initial_guess)
        amp1, mean1, std_dev1, amp2, mean2, std_dev2 = params
        output_matrix[j,i] = amp1,amp2


In [96]:
first_peak = hs.signals.Signal2D(output_matrix[:,:,0])
first_peak.plot()

In [95]:
second_peak = hs.signals.Signal2D(output_matrix[:,:,1])
second_peak.plot()